In [1]:
import pandas as pd
import json

In [2]:
import sys
sys.path.insert(0,'/home/madan/Desktop/joswin_bck/toPendrive/works/analytics_work_bench/nlp-intelligence/analytics_workbench' )

In [3]:
from analytics_workbench.process_text import ProcessText
from analytics_workbench.unsupervised_learning import Unsupervised

In [4]:
import psycopg2
database='crawler_service_test'
user='postgres'
password='postgres'
host='192.168.3.8'

con = psycopg2.connect(database=database, user=user,password=password,host=host)
cursor = con.cursor()

In [5]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in xrange(0, len(seq), size))

In [101]:
def load_data_for_text_processing(cursor,urls,batch_size=2000):
    for batch_urls in chunker(urls,batch_size):
        query = "select all_page_text from (select distinct on (domain) domain,all_page_text from "\
                    "webpage_texts where domain in %s)x"
        cursor.execute(query, (tuple(batch_urls),))
        # batch = cursor.fetchall()
        batch = []
        res = cursor.fetchone()
        while res:
            tt = json.loads(res[0])
            url_text = ''
            for k in tt.keys():
                if 'text_from_meta_base_url' in k :#or 'text_from_urls' in k:
                    url_text = url_text + ' '+tt[k]
            batch.append((url_text,))
            res = cursor.fetchone()
        if not 1.0*len(batch)/batch_size > 0.8:
            continue
        if len(batch[0]) > 1:
            yield pd.DataFrame(batch).fillna('').apply(lambda x: '. '.join(x),1)
        else:
            batch = [i[0] for i in batch]
            yield pd.Series(batch).fillna('')

In [102]:
def get_data_matrix_iter(text_processor,cursor,ecom_urls,batch_size=3500):
    for inp_series in load_data_for_text_processing(cursor,ecom_urls,batch_size):
        yield text_processor.get_matrix_test(inp_series)[0]

In [103]:
query = 'select distinct domain from webpage_texts'
cursor.execute(query)
urls = cursor.fetchall()
urls = [i[0] for i in urls]

In [104]:
len(urls)

5078

In [105]:
data_yielder = load_data_for_text_processing(cursor,urls,batch_size=1016)

In [106]:
text_processor = ProcessText()
text_processor.generate_vectorizer_iter_list(data_yielder,vectorizer_type='Count',synonym_loc=None,
                                stem_type=False,phrase_generation=False,
            stop_words_loc='/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/stopwords.txt',
                lower=True,n_gram_range=(1,2),max_df=0.9,min_df=0.01)

In [107]:
len(text_processor.vocabulary)

378

In [108]:
matrix_iter = get_data_matrix_iter(text_processor,cursor,urls,508)
matrix_list = [i for i in matrix_iter]

In [109]:
from scipy.sparse import hstack,vstack
final_matrix = vstack(matrix_list)

In [110]:
unsup_obj = Unsupervised()
model,mat_transform = unsup_obj.generate_topics(final_matrix,text_processor.vocabulary,'lda',30)

Topic #0:
jewelry*863.17, tools*210.23, silver*205.09, power*155.20, metal*102.85, glass*99.58, fine*96.42, air*89.30, hardware*74.91, gold*71.50
Topic #1:
name*128.53, car*120.55, safety*107.01, needs*97.83, weight*70.33, corporate*67.78, much*67.58, offer*38.84, general*25.60, product*17.47
Topic #2:
software*293.12, web*157.85, website*150.22, phone*142.50, hosting*104.95, ecommerce*104.85, mobile*91.40, site*83.35, design*81.97, selling*78.22
Topic #3:
free*513.36, shoes*360.81, shipping*288.61, free shipping*181.73, sports*134.78, boots*122.98, shoe*118.52, accessories*94.13, school*75.25, orders*74.73
Topic #4:
clothing*516.16, fashion*361.50, design*331.23, vintage*205.60, clothes*172.25, antique*119.05, style*118.72, designed*114.94, collection*108.67, accessories*105.39
Topic #5:
office*166.72, commercial*134.53, including*132.51, chairs*96.10, led*87.28, event*83.88, events*57.79, specializes*53.70, well*8.24, discount*4.67
Topic #6:
gifts*514.53, gift*339.71, since*187.33, m

In [100]:
unsup_obj = Unsupervised()
unsup_obj.generate_clusters(final_matrix,text_processor.vocabulary,5)

No of items in each cluster
0    4996
3       1
2       1
1       1
4       1
dtype: int64
Top terms per cluster:
()
Cluster 0: accessories, contact, new, home, shop, products, order, shipping, product, service
Cluster 1: pack, day, plus, lenses, moist, air, disposable, daily, focus, pro
Cluster 2: full, drum, product, details, full product, product details, new, bass, vintage, effects
Cluster 3: add, cart, oz, fl, skin, care, hair, piece, body, makeup
Cluster 4: unisex, option, inventory, true, price, shopify, title, grateful, product, dead
()


(KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
     n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
     random_state=None, tol=0.0001, verbose=0),
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

#### search option present in page or not

In [88]:
from bs4 import BeautifulSoup
query = "select domain,page_sources ->> 'base_url_source' from webpage_texts offset 70 limit 10"
cursor.execute(query)
res = cursor.fetchone()
while res:
    domain = res[0]
    soup = BeautifulSoup(res[1],'html.parser')
    forms = soup.find_all('form')
    search_present = False
    if forms:
        for form in forms:
            if 'search' in form.get('class','None') or 'search' in form.get('action','None') or \
                        'search' in form.get('role','None'):
                search_present=True
                break
    if not search_present:
        links_in_page = soup.find_all('a')
        if links_in_page:
            for link in links_in_page:
                if not(link.get('href','None').startswith('/') or domain in link.get('href','None')):
                    continue
                if 'search' in link.get('href') or 'search' in link.get_text('None'):
                    search_present = True
                    break
    print(domain,search_present)
    res = cursor.fetchone()

('shopyatra.com', True)
('beyoumultiwear.com', True)
('vossar.com', True)
('mocp.org', True)
('chicagokernel.com', True)
('allchicago247.com', True)
('studioeast6.com', True)
('floralexhibits.com', False)
('silvermoonvintage.com', True)
('mmdstyleboutique.com', True)


#### subscribe option present in page

In [81]:
from bs4 import BeautifulSoup
query = "select domain,page_sources ->> 'base_url_source' from webpage_texts offset 30 limit 10"
cursor.execute(query)
res = cursor.fetchone()
while res:
    domain = res[0]
    soup = BeautifulSoup(res[1],'html.parser')
    links_in_page = soup.find_all('a')
    subscr_present = False
    if links_in_page:
        for link in links_in_page:
            if not(link.get('href','None').startswith('/') or domain in link.get('href','None')):
                continue
            if 'subscrib' in link.get_text('None').lower():
                subscr_present = True
                break
    if not subscr_present:
        buttons = soup.find_all('button')
        if buttons:
            for button in buttons:
                if 'subscrib' in button.get_text('None').lower():
                    subscr_present = True
                    break
    print(domain,subscr_present)
    res = cursor.fetchone()

('theparaders.com', True)
('iheartkeenwah.com', False)
('ilfxm.com', True)
('givesugar.com', False)
('cameraambassador.com', True)
('shopfixture.com', False)
('salveobrands.com', False)
('nocs.se', False)
('theclassicgem.com', False)
('stranddesign.org', False)


#### finding shopify powered websites by looking for shopify link in the website

#### check if website is valid

In [86]:
import re
query = "select domain,page_sources ->> 'base_url_source' from webpage_texts offset 30 limit 10"
cursor.execute(query)
res = cursor.fetchone()
while res:
    domain = res[0]
    soup = BeautifulSoup(res[1],'html.parser')
    valid = True
    base_page_text = soup.get_text('None')
    if re.search('server not found|this shop is currently unavailable',base_page_text,re.I) or \
            re.search('Something went wrong',soup.title.get_text('None'),re.I) :
            valid = False
    print(domain,valid)
    res = cursor.fetchone()

('theparaders.com', True)
('iheartkeenwah.com', True)
('ilfxm.com', True)
('givesugar.com', True)
('cameraambassador.com', True)
('shopfixture.com', True)
('salveobrands.com', False)
('nocs.se', True)
('theclassicgem.com', True)
('stranddesign.org', True)


In [87]:
query = "select domain,page_sources ->> 'base_url_source' from webpage_texts offset 70 limit 10"
cursor.execute(query)
res = cursor.fetchone()
while res:
    domain = res[0]
    soup = BeautifulSoup(res[1],'html.parser')
    valid = True
    base_page_text = soup.get_text('None')
    if re.search('server not found|this shop is currently unavailable',base_page_text,re.I) or \
            re.search('Something went wrong',soup.title.get_text('None'),re.I) :
            valid = False
    print(domain,valid)
    res = cursor.fetchone()

('shopyatra.com', True)
('beyoumultiwear.com', True)
('vossar.com', True)
('mocp.org', True)
('chicagokernel.com', True)
('allchicago247.com', True)
('studioeast6.com', True)
('floralexhibits.com', True)
('silvermoonvintage.com', True)
('mmdstyleboutique.com', True)


In [154]:
query = "select misc_details -> 'shipping_providers' ship1,misc_details -> 'shipping_present_in_page' ship2,"\
        "misc_details -> 'seperate_shipping_page_present' ship3,misc_details -> 'mobile_app_present' mobile_app,"\
        "misc_details -> 'cart_present' cart,misc_details -> 'login_signup_present' login,"\
        "misc_details -> 'all_urls_in_website' from webpage_texts where domain = 'studioeast6.com'"
cursor.execute(query)
res = cursor.fetchall()

In [155]:
tmp = eval(res[0][6])

In [156]:
tmp1 = pd.DataFrame(tmp,columns=['url','url_text'])
tmp1['url'] = tmp1['url'].fillna('').apply(lambda x:x.strip())
tmp1['url_text'] = tmp1['url_text'].fillna('').apply(lambda x:x.strip())
tmp1 = tmp1.drop_duplicates()

In [157]:
tmp1['url_len'] = tmp1['url'].fillna('').apply(lambda x: len(x.strip()))
tmp1['url_text_len'] = tmp1['url_text'].fillna('').apply(lambda x: len(x.strip()))

In [164]:
list(tmp1.sort_values('url_text_len',ascending=False)['url'])[:40]

['http://www.studioeast6.com/pages/designers-we-want-you',
 'http://www.studioeast6.com#2',
 'http://www.studioeast6.com#4',
 'http://www.studioeast6.com#7',
 'http://www.studioeast6.com#6',
 'http://www.studioeast6.com/contact-us',
 'http://www.studioeast6.com#5',
 'http://www.studioeast6.com#8',
 'http://www.studioeast6.com#3',
 'https://www.studioeast6.com/pages/bridal-party',
 'http://www.studioeast6.com/pages/bridal-party',
 'http://www.studioeast6.com/pages/location-and-store-hours',
 'http://www.studioeast6.com/cart',
 'https://www.studioeast6.com/pages/groom',
 'http://www.studioeast6.com#1',
 'http://www.studioeast6.com/pages/terms-and-conditions',
 'http://www.studioeast6.com/pages/delivery-and-returns',
 'http://www.studioeast6.com/collections/shop-our-instagram',
 'https://www.studioeast6.com/pages/bridal',
 'https://www.studio-east6.maharaniweddings.com/',
 'http://www.studioeast6.com/collections/ready-to-ship-anarkalis',
 'http://www.studioeast6.com/collections/suit',
 'h